# Add time serie to the Dataset

In [2]:
import pandas as pd
dt_list = pd.date_range(start='2023-01-21 16:00:00', periods=10000, freq='1450min')
dt_list.to_frame(index=False,name="dateTime")
dt_list.tolist()


df = pd.DataFrame(dt_list)
df.columns=["dateTime"]
df

,dateTime
0,2023-01-21 16:00:00
1,2023-01-22 16:10:00
2,2023-01-23 16:20:00
3,2023-01-24 16:30:00
4,2023-01-25 16:40:00
...,...
9995,2050-08-12 01:50:00
9996,2050-08-13 02:00:00
9997,2050-08-14 02:10:00
9998,2050-08-15 02:20:00


In [3]:
import pandas  as pd
import numpy as np

df1 = pd.read_csv('predictive_maintenance.csv')
result = pd.concat([df1, df], axis=1)
result

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type,dateTime
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,No Failure,2023-01-21 16:00:00
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,No Failure,2023-01-22 16:10:00
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,No Failure,2023-01-23 16:20:00
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,No Failure,2023-01-24 16:30:00
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,No Failure,2023-01-25 16:40:00
...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,M24855,M,298.8,308.4,1604,29.5,14,0,No Failure,2050-08-12 01:50:00
9996,9997,H39410,H,298.9,308.4,1632,31.8,17,0,No Failure,2050-08-13 02:00:00
9997,9998,M24857,M,299.0,308.6,1645,33.4,22,0,No Failure,2050-08-14 02:10:00
9998,9999,H39412,H,299.0,308.7,1408,48.5,25,0,No Failure,2050-08-15 02:20:00


In [4]:
result.to_csv('resultcsv',index=False)

# processing with spark

In [51]:
print(pyspark.__version__)

3.3.1


In [5]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import when,lit
from pyspark import SparkContext
import findspark
import pandas  as pd
findspark.init()
spark = SparkSession.\
builder.\
appName("SendToMongo").\
master('local').\
config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.12:3.0.1').\
getOrCreate()



In [6]:
df_spark = spark.read.csv("resultcsv.csv", header=True, inferSchema=True)
df_spark

DataFrame[UDI: int, Product ID: string, Type: string, Air temperature [K]: double, Process temperature [K]: double, Rotational speed [rpm]: int, Torque [Nm]: double, Tool wear [min]: int, Target: int, Failure Type: string, dateTime: string]

In [7]:
df_spark.show()

+---+----------+----+-------------------+-----------------------+----------------------+-----------+---------------+------+------------+-------------------+
|UDI|Product ID|Type|Air temperature [K]|Process temperature [K]|Rotational speed [rpm]|Torque [Nm]|Tool wear [min]|Target|Failure Type|           dateTime|
+---+----------+----+-------------------+-----------------------+----------------------+-----------+---------------+------+------------+-------------------+
|  1|    M14860|   M|              298.1|                  308.6|                  1551|       42.8|              0|     0|  No Failure|2023-01-21 16:00:00|
|  2|    L47181|   L|              298.2|                  308.7|                  1408|       46.3|              3|     0|  No Failure|2023-01-22 16:10:00|
|  3|    L47182|   L|              298.1|                  308.5|                  1498|       49.4|              5|     0|  No Failure|2023-01-23 16:20:00|
|  4|    L47183|   L|              298.2|                 

In [22]:
df_spark_pd = df_spark.toPandas()
temp_test_list = []
processus_test_list = []
dayTime_test_list1 = []
for index, row in df_spark_pd.iterrows():
    if (row['Air temperature [K]']<297 or row['Air temperature [K]']>298) and  (row['Process temperature [K]']<300 or row['Process temperature [K]']>308):
        print("error",row['Air temperature [K]'],row['Process temperature [K]'], row['dateTime'])
        temp_test_list.append(row['Air temperature [K]'])
        processus_test_list.append(row['Process temperature [K]'])
        dayTime_test_list1.append(row['dateTime'])

df_test_temp=pd.DataFrame(data=temp_test_list,columns=["Air temperature [K]"])
df_test_temp
df_test_process=pd.DataFrame(data=processus_test_list,columns=["Process temperature [K]"])
df_test_process
df_test_time1=pd.DataFrame(data=dayTime_test_list1,columns=["dateTime"])
df_test_time1
result_tmp_test = pd.concat([df_test_temp, df_test_process,df_test_time1], axis=1)
result_tmp_test
result_tmp_test.to_csv('tmp_test.csv',index=False)

error 298.1 308.6 2023-01-21 16:00:00
error 298.2 308.7 2023-01-22 16:10:00
error 298.1 308.5 2023-01-23 16:20:00
error 298.2 308.6 2023-01-24 16:30:00
error 298.2 308.7 2023-01-25 16:40:00
error 298.1 308.6 2023-01-26 16:50:00
error 298.1 308.6 2023-01-27 17:00:00
error 298.1 308.6 2023-01-28 17:10:00
error 298.3 308.7 2023-01-29 17:20:00
error 298.5 309.0 2023-01-30 17:30:00
error 298.4 308.9 2023-01-31 17:40:00
error 298.6 309.1 2023-02-01 17:50:00
error 298.6 309.1 2023-02-02 18:00:00
error 298.6 309.2 2023-02-03 18:10:00
error 298.6 309.2 2023-02-04 18:20:00
error 298.6 309.2 2023-02-05 18:30:00
error 298.6 309.2 2023-02-06 18:40:00
error 298.7 309.2 2023-02-07 18:50:00
error 298.8 309.2 2023-02-08 19:00:00
error 298.9 309.3 2023-02-09 19:10:00
error 298.9 309.3 2023-02-10 19:20:00
error 298.8 309.3 2023-02-11 19:30:00
error 298.9 309.3 2023-02-12 19:40:00
error 299.0 309.4 2023-02-13 19:50:00
error 299.0 309.4 2023-02-14 20:00:00
error 299.0 309.5 2023-02-15 20:10:00
error 299.1 

error 299.1 308.8 2030-01-20 07:00:00
error 299.1 308.8 2030-01-21 07:10:00
error 299.1 308.8 2030-01-22 07:20:00
error 299.1 308.8 2030-01-23 07:30:00
error 299.2 308.9 2030-01-24 07:40:00
error 299.3 309.0 2030-01-25 07:50:00
error 299.2 309.0 2030-01-26 08:00:00
error 299.2 309.0 2030-01-27 08:10:00
error 299.2 309.1 2030-01-28 08:20:00
error 299.2 309.1 2030-01-29 08:30:00
error 299.3 309.2 2030-01-30 08:40:00
error 299.4 309.2 2030-01-31 08:50:00
error 299.4 309.3 2030-02-01 09:00:00
error 299.3 309.2 2030-02-02 09:10:00
error 299.3 309.1 2030-02-03 09:20:00
error 299.3 309.1 2030-02-04 09:30:00
error 299.3 309.1 2030-02-05 09:40:00
error 299.3 309.2 2030-02-06 09:50:00
error 299.3 309.1 2030-02-07 10:00:00
error 299.4 309.1 2030-02-08 10:10:00
error 299.2 309.0 2030-02-09 10:20:00
error 299.3 309.0 2030-02-10 10:30:00
error 299.4 309.1 2030-02-11 10:40:00
error 299.4 309.2 2030-02-12 10:50:00
error 299.3 309.1 2030-02-13 11:00:00
error 299.3 309.1 2030-02-14 11:10:00
error 299.3 

error 302.3 310.7 2034-08-03 17:10:00
error 302.2 310.6 2034-08-04 17:20:00
error 302.2 310.6 2034-08-05 17:30:00
error 302.2 310.6 2034-08-06 17:40:00
error 302.2 310.7 2034-08-07 17:50:00
error 302.3 310.8 2034-08-08 18:00:00
error 302.4 310.9 2034-08-09 18:10:00
error 302.4 310.9 2034-08-10 18:20:00
error 302.4 311.0 2034-08-11 18:30:00
error 302.5 311.0 2034-08-12 18:40:00
error 302.4 311.1 2034-08-13 18:50:00
error 302.4 311.0 2034-08-14 19:00:00
error 302.5 311.0 2034-08-15 19:10:00
error 302.5 311.0 2034-08-16 19:20:00
error 302.4 311.0 2034-08-17 19:30:00
error 302.4 311.1 2034-08-18 19:40:00
error 302.5 311.1 2034-08-19 19:50:00
error 302.5 311.1 2034-08-20 20:00:00
error 302.4 311.0 2034-08-21 20:10:00
error 302.4 311.0 2034-08-22 20:20:00
error 302.3 310.9 2034-08-23 20:30:00
error 302.3 310.9 2034-08-24 20:40:00
error 302.3 310.8 2034-08-25 20:50:00
error 302.3 310.8 2034-08-26 21:00:00
error 302.3 310.9 2034-08-27 21:10:00
error 302.3 310.9 2034-08-28 21:20:00
error 302.2 

error 301.6 311.2 2038-12-21 18:10:00
error 301.5 311.1 2038-12-22 18:20:00
error 301.6 311.1 2038-12-23 18:30:00
error 301.6 311.1 2038-12-24 18:40:00
error 301.6 311.1 2038-12-25 18:50:00
error 301.7 311.1 2038-12-26 19:00:00
error 301.6 311.1 2038-12-27 19:10:00
error 301.7 311.2 2038-12-28 19:20:00
error 301.7 311.1 2038-12-29 19:30:00
error 301.7 311.1 2038-12-30 19:40:00
error 301.8 311.1 2038-12-31 19:50:00
error 301.8 311.1 2039-01-01 20:00:00
error 301.7 311.1 2039-01-02 20:10:00
error 301.7 311.2 2039-01-03 20:20:00
error 301.7 311.3 2039-01-04 20:30:00
error 301.8 311.5 2039-01-05 20:40:00
error 301.8 311.5 2039-01-06 20:50:00
error 301.7 311.2 2039-01-07 21:00:00
error 301.8 311.3 2039-01-08 21:10:00
error 301.8 311.3 2039-01-09 21:20:00
error 301.8 311.3 2039-01-10 21:30:00
error 301.9 311.5 2039-01-11 21:40:00
error 301.9 311.4 2039-01-12 21:50:00
error 301.7 311.1 2039-01-13 22:00:00
error 301.6 311.0 2039-01-14 22:10:00
error 301.5 310.9 2039-01-15 22:20:00
error 301.5 

error 300.2 310.3 2043-01-18 00:30:00
error 300.2 310.4 2043-01-19 00:40:00
error 300.3 310.5 2043-01-20 00:50:00
error 300.3 310.4 2043-01-21 01:00:00
error 300.2 310.3 2043-01-22 01:10:00
error 300.2 310.3 2043-01-23 01:20:00
error 300.2 310.4 2043-01-24 01:30:00
error 300.3 310.4 2043-01-25 01:40:00
error 300.4 310.5 2043-01-26 01:50:00
error 300.4 310.5 2043-01-27 02:00:00
error 300.4 310.5 2043-01-28 02:10:00
error 300.4 310.6 2043-01-29 02:20:00
error 300.4 310.6 2043-01-30 02:30:00
error 300.4 310.6 2043-01-31 02:40:00
error 300.4 310.6 2043-02-01 02:50:00
error 300.4 310.5 2043-02-02 03:00:00
error 300.5 310.5 2043-02-03 03:10:00
error 300.5 310.6 2043-02-04 03:20:00
error 300.4 310.5 2043-02-05 03:30:00
error 300.2 310.4 2043-02-06 03:40:00
error 300.3 310.4 2043-02-07 03:50:00
error 300.2 310.3 2043-02-08 04:00:00
error 300.2 310.3 2043-02-09 04:10:00
error 300.2 310.3 2043-02-10 04:20:00
error 300.2 310.3 2043-02-11 04:30:00
error 300.1 310.3 2043-02-12 04:40:00
error 300.1 

error 298.1 309.3 2047-07-10 07:00:00
error 298.1 309.3 2047-07-11 07:10:00
error 298.1 309.3 2047-07-12 07:20:00
error 298.1 309.2 2047-07-13 07:30:00
error 298.1 309.2 2047-07-14 07:40:00
error 298.1 309.1 2048-06-30 18:00:00
error 298.1 309.0 2048-07-01 18:10:00
error 298.2 309.2 2048-07-02 18:20:00
error 298.2 309.1 2048-07-03 18:30:00
error 298.2 309.1 2048-07-04 18:40:00
error 298.3 309.1 2048-07-05 18:50:00
error 298.3 309.1 2048-07-06 19:00:00
error 298.2 308.9 2048-07-07 19:10:00
error 298.1 308.7 2048-07-08 19:20:00
error 298.1 308.8 2048-07-09 19:30:00
error 298.2 308.8 2048-07-10 19:40:00
error 298.2 308.8 2048-07-11 19:50:00
error 298.1 308.7 2048-07-12 20:00:00
error 298.1 308.7 2048-07-13 20:10:00
error 298.2 308.7 2048-07-14 20:20:00
error 298.3 308.8 2048-07-15 20:30:00
error 298.3 308.8 2048-07-16 20:40:00
error 298.3 308.9 2048-07-17 20:50:00
error 298.3 308.9 2048-07-18 21:00:00
error 298.3 308.9 2048-07-19 21:10:00
error 298.3 309.1 2048-07-20 21:20:00
error 298.3 

In [14]:
df_spark_pd = df_spark.toPandas()
Rotational_test_list = []
Torque_test_list = []
Tool_test_list = []
dayeTime_test_list = []
for index, row in df_spark_pd.iterrows():
    if (row['Rotational speed [rpm]']<1300 or row['Rotational speed [rpm]']>1811) and (row['Torque [Nm]']<19 or row['Torque [Nm]']>50) and (row['Tool wear [min]']<0 or row['Tool wear [min]']>216):
        print("error",row['Rotational speed [rpm]'],row['Torque [Nm]'],row['Tool wear [min]'], row['dateTime'])
        Rotational_test_list.append(row['Rotational speed [rpm]'])
        Torque_test_list.append(row['Torque [Nm]'])
        Tool_test_list.append(row['Tool wear [min]'])
        dayeTime_test_list.append(row['dateTime'])
                                
                                
df_test_R=pd.DataFrame(data=Torque_test_list,columns=["Rotational speed [rpm]"])
df_test_R
df_test_T=pd.DataFrame(data=Torque_test_list,columns=["Torque [Nm]"])
df_test_T
df_test_K=pd.DataFrame(data=Tool_test_list,columns=["Tool wear [min]"])
df_test_K
df_test_time=pd.DataFrame(data=dayeTime_test_list,columns=["dateTime"])
df_test_time
result_test = pd.concat([df_test_R, df_test_T,df_test_K,df_test_time], axis=1)
result_test
result_test.to_csv('V_test.csv',index=False)

error 2143 17.3 227 2034-03-02 15:40:00
error 1298 65.5 229 2035-02-21 02:30:00
error 1238 54.6 226 2035-11-08 21:40:00
error 2168 17.7 219 2036-07-24 16:30:00
error 1262 70.5 234 2037-12-05 03:00:00
error 2097 18.9 232 2044-04-01 01:10:00
error 2271 16.2 218 2049-12-16 10:20:00


# Save data in Mongodb

In [ ]:
result_tmp_test.write\
    .format('com.mongodb.spark.sql.DefaultSource')\
    .option("database", "predict_maintenance").option("collection","tmp") \
    .save()


In [ ]:
result_test.write\
    .format('com.mongodb.spark.sql.DefaultSource')\
    .option("database", "predict_maintenance").option("collection","v") \
    .save()
